# getSensitivityIDF

The following code outputs a number of objects.
<br>
<br>
<u><b>IDF for jEplus:</b></u>
<br>
There are a total of 1573 parameters which need to be analysed for the sensitivity analysis. Manually adding the appropriate tags into the IDF so that jEplus knows which parameters to analyse is infeasible. This script takes in the list of parameters from the uncertainty analysis code as well as the building IDF and returns a new IDF with the relevent parameter tags.
<br>
<br>
<u><b>Parameter CSV for jEplus:</b></u>
<br>
jEplus also requires the parameters which are to be analysed with their respective minimum and maximum values to be defined. This script returns a CSV file listing these parameters which can be directly put into jEplus.
<br>
<br>
<u><b>Jobs CSV for jEplus:</b></u>
<br>
jEplus requires us to specify the jobs which will be run. This code outputs a CSV file listing all the jobs to be run which can be put into jEplus.
<br>
<br>
<u><b>Parameter JSON File:</b></u>
<br>
A JSON file which will be used later in the <i>"SensitivityAnalysis.ipynb"</i> notebook is also output by this script.
<br>
<br>
<u><b>Extremes Jobs CSV:</b></u>
<br>
During the calibration process it is possible that a number of parameters will be changed to their maximum/minimum value. To ensure that no errors occur in EnergyPlus when such an IDF is used, it is important to run a simulation where every value is set to an extreme (minimum/maximum). This script outputs another jobs CSV file where each parameter is set to a maximum and minimum value.

In [1]:
import pandas as pd
import re
import numpy as np

class JEPlusSensitivityAnalysis():
    
    def __init__(self, idfFile, iddFile, paramFile):
        self.paramFile = paramFile
        self.idfFile = idfFile
        self.iddFile = iddFile
        
        self.idf, self.idd = self.getIDFandIDD(self.idfFile, self.iddFile)
        
        self.full_df = pd.read_csv(paramFile).dropna()
        self.initial_df = pd.read_csv(paramFile).loc[:, ['Class', 'Object', 'Field']].dropna()
        self.df = self.addTagNumbers(self.initial_df)
        self.paramsCSV = self.initial_df.copy()
        
        self.uniqueClasses = self.df['Class'].unique()
        self.classDict = self.getClassDict()
        
        self.tagNumSanityCheck = []
        
        self.createJEPlusIDF()
        self.jEplusParameters = self.createParameterCSV()
        self.jEplusJobs, self.jobsDict, self.minimax_jobs_df = self.createJobsCSVandJSON()
        
        
    def getIDFandIDD(self, idfFile, iddFile):
        idf = open(idfFile, "r")
        initial_idf_lines = idf.readlines()
        
        initial_idf_copy = []
        for line in initial_idf_lines:
            newLine = line.strip()
            initial_idf_copy.append(newLine)
            
        idf_lines = self.removeIDFcomments(initial_idf_copy)
    
        idd = open(iddFile, "r")
        idd_lines = idd.readlines()
    
        return idf_lines, idd_lines
    
    
    def extractFields(self, start):
        classList = []
        num = start
        while self.idd[num] != '\n':
            if self.idd[num].strip()[0] == 'A' or self.idd[num].strip()[0] == 'N':
                pattern = re.compile(r'\\field')
                matches = pattern.search(self.idd[num])
                
                if matches is not None:
                    classList.append(self.idd[num].strip().split('\\field ')[1])
            
            num = num + 1
        
        return classList
    
    
    def getClassDict(self):
        classDict = {}
        for uniqueClass in self.uniqueClasses:
            i = 0
            while i < len(self.idd):
                line = self.idd[i]
                adjLine = line.strip().split(',')[0]
            
                if adjLine == uniqueClass:
                    currentList = self.extractFields(i)
                    classDict[uniqueClass] = currentList
                    break
            
                i = i + 1
                
        return classDict
    
    
    def addTagNumbers(self, df):
        dfCopy = df.copy()
        tagNums = []
        i = 0
        while i < len(dfCopy):
            tagNums.append(i)
            i = i + 1
            
        dfCopy['Tag Number'] = tagNums
        
        return dfCopy
    
    
    def removeIDFcomments(self, idf):
        idfCopy = []
        
        for line in idf:
            if len(line) == 0:
                idfCopy.append(line)
        
            elif line.strip()[0] != '!':
                idfCopy.append(line)
                
        if len(idfCopy[0]) == 0:
            del idfCopy[0]
            
        return idfCopy
    
    
    def removeComment(self, line):
        newLine = re.split('!', line)[0].strip()
        
        return newLine
    
    
    def getClassObject(self, start):
        num = start
        line = self.idf[num]
        pattern = re.compile(r';')
        matches = None
        classFields = []
        
        while matches is None:
            lineWithoutComment = self.removeComment(self.idf[num])
            currentLine = re.split(',|;', lineWithoutComment)
            del currentLine[-1]
            
            for element in currentLine:
                classFields.append(element)
            
            matches = pattern.search(self.idf[num])
            num = num + 1
        
        classObjectName = classFields[1]
        return classObjectName
    
    
    def oneObjectTagAdd(self, start, uniqueClass):
        num = start
        pattern = re.compile(r';')
        matches = None
        searchTerms = []
        searchTermsDict = {}
        elementsPerLine = []
    
        while matches is None:
            count = 0
            lineWithoutComment = self.removeComment(self.idf[num])
            currentLine = re.split(',|;', lineWithoutComment)
            del currentLine[-1]
                    
            matches = pattern.search(self.idf[num])
            for element in currentLine:
                searchTerms.append(element.strip())
                count = count + 1
            
            elementsPerLine.append(count)
            num = num + 1
            
        assert searchTerms[0] == uniqueClass, 'Class Mismatch'
    
        del searchTerms[0]
    
        searchTermsDict[uniqueClass] = searchTerms
        searchTermsDict['ElementsPerLine'] = elementsPerLine
    
        assert sum(searchTermsDict['ElementsPerLine']) == len(searchTermsDict[uniqueClass]) + 1
        
        currentDF = self.df.loc[self.df['Class'] == uniqueClass, :]
        num = start
    
        for f in currentDF['Field'].unique():
            if f not in self.classDict[uniqueClass]:
                print(f'{f} not in {uniqueClass} classDict')
    
        pat = re.compile('@@')
        for val in self.classDict[uniqueClass]:
                
            if val in currentDF['Field'].unique():
                currentIndex = self.classDict[uniqueClass].index(val)
                tagNum = currentDF.loc[currentDF['Field'] == val, ['Tag Number']].values[0][0]
                
                i = 0
                j = 0
                while i < currentIndex + 2:
                    
                    lineNow = self.idf[num + j]
                    
                    i = i + searchTermsDict['ElementsPerLine'][j]
                    j = j + 1
                
                tagged = pat.search(lineNow)
                if tagged is None:
                    new = new = lineNow.replace(searchTermsDict[uniqueClass][currentIndex], f'@@tag{tagNum}@@', 1)
                    
                elif tagged is not None:
                    lineArray = lineNow.split('@@')
                    newLineNow = lineArray[-1]
                    del lineArray[-1]
                    newt = newLineNow.replace(searchTermsDict[uniqueClass][currentIndex], f'@@tag{tagNum}@@', 1)
                    new = '@@'.join(lineArray) + '@@' + newt

                self.idf[num + j - 1] = new
                self.tagNumSanityCheck.append(tagNum)
                
    
    def multipleObjectTagAdd(self, start, uniqueClass, uniqueObject):
        num = start
        pattern = re.compile(r';')
        matches = None
        searchTerms = []
        searchTermsDict = {}
        elementsPerLine = []
        
        while matches is None:
            count = 0
            lineWithoutComment = self.removeComment(self.idf[num])
            currentLine = re.split(',|;', lineWithoutComment)
            del currentLine[-1]
                    
            matches = pattern.search(self.idf[num])
            for element in currentLine:
                searchTerms.append(element.strip())
                count = count + 1
                
            elementsPerLine.append(count)
            num = num + 1
            
        assert searchTerms[0] == uniqueClass, 'Class Mismatch'
        assert searchTerms[1] == uniqueObject, 'Object Mismatch'
        
        del searchTerms[0:2]
        
        searchTermsDict[f'{uniqueClass} --> {uniqueObject}'] = searchTerms
        searchTermsDict['ElementsPerLine'] = elementsPerLine
        
        assert sum(searchTermsDict['ElementsPerLine']) == len(searchTermsDict[f'{uniqueClass} --> {uniqueObject}']) + 2
        
        currentDF = self.df.loc[(self.df['Class'] == uniqueClass) & (self.df['Object'] == uniqueObject), :]
        num = start
        
        for f in currentDF['Field'].unique():
            if f not in self.classDict[uniqueClass]:
                print(f'{f} not in {uniqueClass} classDict')
        
        pat = re.compile('@@')
        for val in self.classDict[uniqueClass]:
            if val not in currentDF['Field'].unique():
                x = 0
                
            if val in currentDF['Field'].unique():
                currentIndex = self.classDict[uniqueClass].index(val)
                tagNum = currentDF.loc[currentDF['Field'] == val, ['Tag Number']].values[0][0]
                stringedTagNum = str(tagNum)
                
                i = 0
                j = 0
    
                while i < currentIndex + 2:
                    
                    lineNow = self.idf[num + j]
                    
                    i = i + searchTermsDict['ElementsPerLine'][j]
                    j = j + 1
                
                tagged = pat.search(lineNow)
                if tagged is None:
                    new = lineNow.replace(searchTermsDict[f'{uniqueClass} --> {uniqueObject}'][currentIndex-1], f'@@tag{tagNum}@@', 1)
                    
                elif tagged is not None:
                    lineArray = lineNow.split('@@')
                    newLineNow = lineArray[-1]
                    del lineArray[-1]
                    newt = newLineNow.replace(searchTermsDict[f'{uniqueClass} --> {uniqueObject}'][currentIndex-1], f'@@tag{tagNum}@@', 1)
                    new = '@@'.join(lineArray) + '@@' + newt

                self.idf[num + j - 1] = new
                self.tagNumSanityCheck.append(tagNum)
       
    
    def createJEPlusIDF(self):
        pattern = re.compile(r';')
        matches = None
        
        for uniqueClass in self.uniqueClasses:
            currentDF = self.df.loc[self.df['Class'] == uniqueClass, :]
            numOfObjInClasses = len(currentDF['Object'].unique())
            
            if numOfObjInClasses == 1:
                i = 0
                while i < len(self.idf):
                    line = self.idf[i]
                    adjLine = line.split(',')[0]
                    
                    if len(adjLine) > 0:
                        isClass = 1
                    else:
                        isClass = 0
                    
                    if adjLine not in self.uniqueClasses and isClass == 1:
                        pattern = re.compile(';')
                        matches = None
                        
                        while matches is None:
                            line = self.idf[i]
                            matches = pattern.search(line)
                            if matches is not None:
                                    break
                            i = i + 1
                        
                    if adjLine == uniqueClass:
                        self.oneObjectTagAdd(i, uniqueClass)
                        break
                    
                    i = i + 1
                    
            if numOfObjInClasses != 1:
                uniqueObjects = currentDF['Object'].unique()
                
                for uniqueObject in uniqueObjects:
                    newCurrentDF = currentDF.loc[(currentDF['Class'] == uniqueClass) & (currentDF['Object'] == uniqueObject), :]
                    j = 0
                    
                    while j < len(self.idf):
                        line = self.idf[j]
                        adjLine = line.split(',')[0]
                        
                        if len(adjLine) > 0:
                            isClass = 1
                        else:
                            isClass = 0
                        
                        if adjLine not in self.uniqueClasses and isClass == 1:
                            pattern = re.compile(';')
                            matches = None
                    
                            while matches is None:
                                line = self.idf[j]
                                matches = pattern.search(line)
                                if matches is not None:
                                    break
                                j = j + 1
                        
                        if adjLine == uniqueClass:
                            uo = self.getClassObject(j).strip()
                        
                        if adjLine == uniqueClass and uo == uniqueObject:
                            self.multipleObjectTagAdd(j, uniqueClass, uniqueObject)
                            break
                            
                        j = j + 1
                        
                    
    def createParameterCSV(self):
        tagNums = []
        ids = []
        names = []
        paramTypes = []
        descriptions = []
        searchStrings = []
        selectedValueIndexes = []
        valueTypes = []
        valueStrings = []
        
        default_col_num = self.full_df.columns.get_loc('Default')
        min_col_num = self.full_df.columns.get_loc('Minimum')
        max_col_num = self.full_df.columns.get_loc('Maximum')
        
        i = 0
        while i < len(self.full_df):
            tagNums.append(i)
            
            ids.append('P' + str(i))
            
            Class = self.full_df['Class'][i]
            obj = self.full_df['Object'][i]
            field = self.full_df['Field'][i]
            
            names.append(obj + ' ' + field)
            
            paramTypes.append('PARAMETRICS')
            
            descriptions.append(Class + ' - ' + obj + ' ' + field)
            
            searchStrings.append('@@tag' + str(i) + '@@')
            
            selectedValueIndexes.append(0)
            
            valueTypes.append('DOUBLE')
            
            val1 = self.full_df.iloc[i, default_col_num]
            val2 = self.full_df.iloc[i, min_col_num]
            val3 = self.full_df.iloc[i, max_col_num]
    
            valueStrings.append(f'{{{val1}, {val2}, {val3}}}')
            
            i = i + 1
            
        SAparams = pd.DataFrame({'# ID': ids, 'Name': names, 'Parameter Type': paramTypes, 'Description': descriptions,
                         'Search String': searchStrings, 'Value Type': valueTypes, 'Value String': valueStrings,
                        'Selected Value Index': selectedValueIndexes})
        
        return SAparams
    
    
    def createJobsCSVandJSON(self):  
        the_list = [self.full_df['Default'].values.copy()]
        max_jobs_list = []
        min_jobs_list = []
        jobsDict = {}
        minJobNum = 2
        maxJobNum = 3
        for index, row in self.full_df.iterrows():
            default = row['Default']
            maximum = row['Maximum']
            minimum = row['Minimum']
    
            def_vals = self.full_df['Default'].values.copy()
            max_vals = self.full_df['Default'].values.copy()
            min_vals = self.full_df['Default'].values.copy()
    
            max_vals[index] = maximum
            min_vals[index] = minimum
            
            max_jobs_list.append(maximum)
            min_jobs_list.append(minimum)
    
            the_list.append(list(min_vals))
            the_list.append(list(max_vals))
            
            theClass = row['Class']
            theObject = row['Object']
            theField = row['Field']
            
            name = theClass + '-->' + theObject + '-->' + theField
            jobsDict[name] = {}
            jobsDict[name]['Default'] = default
            jobsDict[name]['MinJobNum'] = minJobNum
            jobsDict[name]['Min'] = minimum
            jobsDict[name]['MaxJobNum'] = maxJobNum
            jobsDict[name]['Max'] = maximum
            
            minJobNum = minJobNum + 2
            maxJobNum = maxJobNum + 2
            
        jobs_df = pd.DataFrame(np.array(the_list))
        minimax_jobs_df = pd.DataFrame([min_jobs_list, max_jobs_list])
        
        jobs_df.insert(0, "Job Name", 'Job', True)
        jobs_df.insert(1, "IDF File", 0, True)
        jobs_df.insert(2, "Weather File", 0, True)
        
        minimax_jobs_df.insert(0, "Job Name", 'Job', True)
        minimax_jobs_df.insert(1, "IDF File", 0, True)
        minimax_jobs_df.insert(2, "Weather File", 0, True)
        
        return jobs_df, jobsDict, minimax_jobs_df

In [2]:
idfFile = "..\IDF_Files\Building.idf"
iddFile = "..\IDD_Files\V85_Energy+.idd"
paramFile = "../UncertaintyAnalysis/Filtered_TuneableParameters.csv"

jeplus = JEPlusSensitivityAnalysis(idfFile, iddFile, paramFile)

writeIDF = False
IDFname = 'Building_IDF4JEPlus.idf'

writeJEPlusParams = False
JEPlusParamsName = 'Building_Parmas4JEPlus.csv'

writeJEPLusJobsCSV = False
JEPlusJobsCSVName = 'Building_Jobs4JEPlus.csv'

writeJEPLusJobsJSON = False
JEPlusJobsJSONName = 'Building_JSON_Jobs.json'

writeMinimaxJobsCSV = False
MinimaxJobsCSVName = 'BuildingMinimaxJobs.csv'

In [3]:
print(f'{len(jeplus.tagNumSanityCheck)} tags have been added to the IDF!')

1573 tags have been added to the IDF!


In [4]:
k = 0
l = 0

while k < len(jeplus.full_df):
    if k not in jeplus.tagNumSanityCheck:
        print(f'@@tag{k}@@ could not be added!')
        l = 1
        
    k = k + 1
    
if l == 0:
    print('All tags have been added. None are missing!')

All tags have been added. None are missing!


In [5]:
import json

if writeIDF:
    f = open(IDFname, "a")
    for l in jeplus.idf:
        f.writelines(l + '\n')
        
    f.close()
    
if writeJEPlusParams:
    jeplus.jEplusParameters.to_csv(JEPlusParamsName, index=False)
    
if writeJEPLusJobsCSV:
    jeplus.jEplusJobs.to_csv(JEPlusJobsCSVName, index=False, header=False)
    
if writeJEPLusJobsJSON:
    with open(JEPlusJobsJSONName, 'w') as json_file:
        json.dump(jeplus.jobsDict, json_file)
    
    json_file.close()
    
if writeMinimaxJobsCSV:
    jeplus.minimax_jobs_df.to_csv(MinimaxJobsCSVName, index=False, header=False)